# TO DO!

- add a comparisons df!
- figure out how to deal with the fact that data is not great!

## required packages

In [1]:
require(vegan)
require(hypervolume)
require(dplyr)
require(ggplot2)
source("00_hv_functions.R")

Loading required package: vegan
Loading required package: permute
Loading required package: lattice
This is vegan 2.5-2
Loading required package: hypervolume
Loading required package: Rcpp
Loading required package: rgl
Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”Warning message:
“'rgl_init' failed, running with rgl.useNULL = TRUE”Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: ggplot2


## read in data

In [2]:
#trees_df   = read.csv("../Results/trees_family_matrix.csv", row.names=1)
trees_df   = read.csv("../Results/trees_genus_matrix.csv", row.names=1)
#trees_df   = read.csv("../Results/trees_matrix.csv", row.names=1)

mammals_df = read.csv("../Results/mammals_matrix.csv", row.names=1)

beetles_df = read.csv("../Results/beetles_matrix.csv", row.names=1)

## pca function

- do I want to do relative abundance?
    - my thought is no as absolute numbers matter in the analysis?
- what about scale?
    - yes because pca requires it?

In [3]:
setClass("pca.out", slots = c(tot.chi = "numeric",
                              exp.var = "numeric",
                              axis    = "data.frame",
                              CA      = "list",
                              Ybar    = "matrix",
                              inertia = "character"))

do_pca = function(df, scale = TRUE, plot = TRUE){
    
    # if i did want to do relative abundance?
    # df = apply(df, 1, function(x) x/sum(x))

    df.pca = rda(df, scale = scale)
    
    out = new("pca.out", tot.chi = df.pca$tot.chi,
                         exp.var = head(df.pca$CA$eig/df.pca$tot.chi, 10),
                         axis    = as.data.frame(df.pca$CA$u),
                         CA      = df.pca$CA,
                         Ybar    = df.pca$Ybar,
                         inertia = df.pca$inertia)
    
    out@axis$plot    = unlist(strsplit(rownames(out@axis), "_"))[ c(T,F,F)]
    out@axis$subplot = unlist(strsplit(rownames(out@axis), "_"))[ c(F,T,F)]
    out@axis$census  = unlist(strsplit(rownames(out@axis), "_"))[ c(F,F,T)]
    
    
    if (plot == TRUE){
        biplot(df.pca, scaling = -1)
    }
    
    return(out)
    
}

In [37]:
pca.rslt_trees   = do_pca(trees_df, scale = F, plot = F)
cat("Explained Varience Trees:")
pca.rslt_trees@exp.var

pca.rslt_mammals = do_pca(mammals_df, scale = F, plot = F)
cat("Explained Varience Mammals:")
pca.rslt_mammals@exp.var

pca.rslt_beetles = do_pca(beetles_df, scale = F, plot = F)
cat("Explained Varience Beetles:")
pca.rslt_beetles@exp.var

Explained Varience Trees:

PC1        PC2        PC3        PC4        PC5        PC6        PC7 
0.02370149 0.02068754 0.02002843 0.01924347 0.01856575 0.01788064 0.01684064 
       PC8        PC9       PC10 
0.01606711 0.01579577 0.01495563

Explained Varience Mammals:

PC1        PC2        PC3        PC4        PC5        PC6        PC7 
0.41494884 0.21461567 0.09989944 0.07337631 0.06039596 0.03230368 0.02925390 
       PC8        PC9       PC10 
0.01902081 0.01543136 0.01221410

Explained Varience Beetles:

PC1          PC2          PC3          PC4          PC5          PC6 
0.8751009364 0.0844253550 0.0180393577 0.0062422800 0.0048188174 0.0030864325 
         PC7          PC8          PC9         PC10 
0.0022513596 0.0013391250 0.0007822565 0.0006395838

## hypervolumes function

In [67]:
setClass("hv.rslts", slots = c(hvlist = "list",
                               rslts  = "data.frame"))

# function to make hypervolumes for all..
hvs_rslts = function(df){
    
  names = c()
  for (p in unique(df$plot)){ for (c in unique(df$census)){
    names = c(names, paste0(p, "_", c))
  }}

  hvlist = list()
    
  rslts           = data.frame(matrix(NA, nrow = length(names), ncol = 12))
  colnames(rslts) = c("plot", "census", "centroid_PC1", "centroid_PC2",
                    "centroid_PC3", "volume", "PC1_l", "PC1_h", "PC2_l",
                    "PC2_h", "PC3_l", "PC3_h")
  rownames(rslts) = names
  rslts$plot_c = names
       
  for (i in names){                                                                                                                                                                                            

    p = unlist(strsplit(i, "_"))[[1]]
    c = unlist(strsplit(i, "_"))[[2]]
      
    tmp = subset(df, plot == p & census == c, select = c("PC1", "PC2", "PC3"))
    tmp = scale(tmp, center = T, scale = T)
  
    if (nrow(tmp) < 20){
      hv = NA
    } else{
      hv = hypervolume_gaussian(tmp, name = i, verbose = FALSE)
    }
    
    hvlist[[i]] = hv
      
    rslts[i, "plot"]     = unlist(strsplit(i, "_"))[1]
    rslts[i, "census"]   = unlist(strsplit(i, "_"))[2]

    if (class(hv) == "Hypervolume"){

      rslts[i, "centroid_PC1"] = get_centroid(hv)[1]
      rslts[i, "centroid_PC2"] = get_centroid(hv)[2]
      rslts[i, "centroid_PC3"] = get_centroid(hv)[3]

      rslts[i, "volume"]       = hv@Volume

      rslts[i, "PC1_l"]        = min(hv@RandomPoints[,1])
      rslts[i, "PC1_h"]        = max(hv@RandomPoints[,1])
      rslts[i, "PC2_l"]        = min(hv@RandomPoints[,2])
      rslts[i, "PC2_h"]        = max(hv@RandomPoints[,2])
      rslts[i, "PC3_l"]        = min(hv@RandomPoints[,3])
      rslts[i, "PC3_h"]        = max(hv@RandomPoints[,3])

    }     
      
  }
  
  out = new("hv.rslts", hvlist = hvlist, rslts = rslts)
    
  return(out)
}                              

In [68]:
hvs.rslts_trees   = hvs_rslts(pca.rslt_trees@axis)
hvs.rslts_mammals = hvs_rslts(pca.rslt_mammals@axis)
hvs.rslts_beetles = hvs_rslts(pca.rslt_beetles@axis)

cat("Trees: ",   sum(is.na(hvs.rslts_trees@rslts$centroid_PC1))/nrow(hvs.rslts_trees@rslts), "\n")
cat("Mammals: ", sum(is.na(hvs.rslts_mammals@rslts$centroid_PC1))/nrow(hvs.rslts_mammals@rslts), "\n")
cat("Beetles: ", sum(is.na(hvs.rslts_beetles@rslts$centroid_PC1))/nrow(hvs.rslts_beetles@rslts))

Trees:  0.1666667 
Mammals:  1 
Beetles:  0.3833333

In [ ]:
attributes <- read.csv("../Results/trees_sorted.csv")
a = as.data.frame(attributes %>% group_by(plot_c) %>% summarise(stem_C = sum(stem_C)))

hvs.rslts@rslts = merge(hvs.rslts@rslts, a, by = "plot_c", all.x = T)